In [1]:
from sys import stdout
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

import shap
  
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 

from progressbar import ProgressBar

In [ ]:
#import training datadet
inter = pd.ExcelFile('training data_Github.xlsx')
inter = pd.read_excel(inter,'Combined')
#print(inter.info())
comb = inter.dropna(axis=0)
#print(comb.info())
comb = comb[comb["tR (min)"] != "?"]
#preprocess
for col in comb.columns[1:]:
    if (comb[col] == 0).all():
        comb = comb.drop([col], axis = 1)
comb = comb.reset_index(drop=True)
print(comb.info())

In [3]:
#PLS portion


Y = {'tR (min)': [round(x,4) for x in comb['tR (min)']]}
Y = pd.DataFrame(comb, columns = ['tR (min)'])
#print(comb.info())
X = comb.drop(['Project','Column','Compound','tR (min)'], axis=1)

### Standardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(Y)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
Y=TargetVarScalerFit.transform(Y)

X = pd.DataFrame(X)
Y = pd.DataFrame(Y)
print(X.info())
print(Y.info())

#PLS Regression
n_comp = 5
pls = PLSRegression(n_components=n_comp)
# Cross-validation
pls.fit(X, Y)

# Define a wrapper function for PLS model predictions
def pls_predict(X):
    return pls.predict(X).flatten()  # Flatten the predictions if necessary

X = np.asarray(X).astype('float32')
#print(X)
explainer = shap.KernelExplainer(pls_predict, X)
shap_values = explainer.shap_values(X)

import sys
size = sys.getsizeof(shap_values)
import csv
with open("shapley_PLS_re.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    
    writer.writerows(shap_values)

Using 1790 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Columns: 141 entries, 0 to 140
dtypes: float64(141)
memory usage: 1.9 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1790 non-null   float64
dtypes: float64(1)
memory usage: 14.1 KB
None


  0%|          | 0/1790 [00:00<?, ?it/s]